# Practica 1

Participantes:
- Daniel López Escobar
- Ángel Gallego Muñoz
- Raúl Durán Catalán

## Cargamos datos

In [7]:
import pandas as pd
data_raw = pd.read_csv("smart_agriculture_bangladesh.csv")
print(len(data_raw))

10000


Eliminamos los datos nulos y los duplicados. Vemos que nuestro dataset no tiene ni casillas vacias ni filas duplicadas.

In [8]:
old_n_rows = len(data_raw)
# Eliminar nulos
data = data_raw.dropna().drop_duplicates()
new_n_rows = len(data)

dif_n_rows = old_n_rows - new_n_rows 
# Como da 0, no se han eliminado ningunas
dif_n_rows

0

Pasamos el timestamp de string a entero para poder trabajar con él mejor, ya que con numeros podemos comparar facilmente y operarar mejor.

In [9]:
data['timestamp_dt'] = pd.to_datetime(data['timestamp'])
data = data.drop(columns=["timestamp"])

## Analizamos y transformamos los datos

In [10]:
# Guardamos el nombre de las columnas de nuestro dataset, nos permitirá iterar y conocer mejor el dataset
nominal_categories = list(data.select_dtypes(exclude=["object", "str"]).columns.tolist())
categorical_categories = list(data.select_dtypes(include=["object", "str"]).columns.tolist())

print(f"Las categoricas nominales de los datos son: {nominal_categories}")
print(f"Las categoricas categóricas de los datos son: {categorical_categories}")

Las categoricas nominales de los datos son: ['temperature', 'humidity', 'rainfall', 'soil_moisture', 'soil_ph', 'light_intensity', 'fertilizer_used', 'irrigation_needed', 'yield_estimate', 'anomaly_flag', 'timestamp_dt']
Las categoricas categóricas de los datos son: ['device_id', 'location', 'crop_type', 'season', 'crop_health', 'pest_risk']


Al describir las variables nominales nos ayuda a comprender mejor los datos con los que trabajamos. Entre otras cosas nos ayuda a hacernos una idea de la magnitud de cada característica y cómo se distribuyen sus valores (si están dispersos o no...)

In [11]:
# Datos nominales
data.describe()

,temperature,humidity,rainfall,soil_moisture,soil_ph,light_intensity,fertilizer_used,irrigation_needed,yield_estimate,anomaly_flag,timestamp_dt
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000
mean,28.618411,66.692239,3.019284,47.019202,6.380495,12188.731100,30.521662,0.181500,13.134026,0.020200,2024-03-30 23:17:08.880000
min,15.000000,40.000000,0.000000,10.000000,5.500000,0.000000,0.000000,0.000000,1.120000,0.000000,2024-01-01 00:00:00
25%,23.520000,61.070000,0.000000,37.560000,6.020000,42.000000,16.280000,0.000000,3.040000,0.000000,2024-02-14 23:45:00
50%,30.150000,67.010000,0.000000,46.660000,6.380000,97.000000,29.820000,0.000000,4.450000,0.000000,2024-03-30 23:30:00
75%,32.890000,72.380000,0.000000,56.142500,6.740000,26307.500000,43.262500,0.000000,25.590000,0.000000,2024-05-14 23:15:00
max,42.000000,95.000000,50.000000,95.000000,8.090000,50000.000000,101.130000,1.000000,38.500000,1.000000,2024-06-28 23:00:00
std,5.399040,8.178572,8.151775,13.748611,0.502964,15317.903602,18.900354,0.385451,12.929829,0.140691,NaN


Al ver la correlacion vemos que no tienen demasiada correlación las variables nominales. El valor mas alto es el 0.71 entre temperatura y tiempo. Muy posiblemente porque nuestro dataset va desde 1 de enero de 2024 al 28 de junio de 2024, por lo que la tempetura tiene un patron de crecimiento al ir desde invierno hasta verano.

In [12]:
data[nominal_categories].corr()

,temperature,humidity,rainfall,soil_moisture,soil_ph,light_intensity,fertilizer_used,irrigation_needed,yield_estimate,anomaly_flag,timestamp_dt
temperature,1.000000,0.504125,0.190840,-0.501244,0.025539,-0.020302,0.002791,0.346911,-0.074165,0.007173,0.716783
humidity,0.504125,1.000000,0.131522,-0.237340,0.002026,-0.019534,0.002478,0.157241,-0.027066,0.010276,0.463975
rainfall,0.190840,0.131522,1.000000,0.358400,-0.008890,-0.128521,-0.023368,-0.164025,0.016682,-0.017245,0.168061
soil_moisture,-0.501244,-0.237340,0.358400,1.000000,-0.018479,-0.047018,-0.011701,-0.656259,0.099559,-0.010514,-0.348846
soil_ph,0.025539,0.002026,-0.008890,-0.018479,1.000000,-0.003866,-0.025306,0.009983,-0.210766,-0.006049,0.032952
light_intensity,-0.020302,-0.019534,-0.128521,-0.047018,-0.003866,1.000000,0.014070,0.033193,0.002825,0.010986,-0.020140
fertilizer_used,0.002791,0.002478,-0.023368,-0.011701,-0.025306,0.014070,1.000000,0.011878,-0.000523,-0.010602,0.000451
irrigation_needed,0.346911,0.157241,-0.164025,-0.656259,0.009983,0.033193,0.011878,1.000000,-0.122815,0.002466,0.231035
yield_estimate,-0.074165,-0.027066,0.016682,0.099559,-0.210766,0.002825,-0.000523,-0.122815,1.000000,-0.008737,-0.041213
anomaly_flag,0.007173,0.010276,-0.017245,-0.010514,-0.006049,0.010986,-0.010602,0.002466,-0.008737,1.000000,0.009382


Al analizar los valores categóricos vemos que salvo `device_id` el resto tienen pocos valores. `device_id` al tener tantos valores igual no nos merece la pena pasarlo a OneHot Encoding, ya que al tener tantos valores, acabaremos con mas columnas de `device_id` que el resto de características, y convertiremos nuestros datos en una matriz demasiado dispersa. Por ello, vamos a eliminarla, nos será de uso posteriormente para obtener mas informacion del dato en concreto posteriormente pudiendo referencia en el dataframe inicial.

In [13]:
# Datos categoricos
data.describe(include=["object", "str"])

,device_id,location,crop_type,season,crop_health,pest_risk
count,10000,10000,10000,10000,10000,10000
unique,48,6,5,2,3,2
top,BD-RAJ-007,Chattogram,Wheat,summer,Healthy,Low
freq,240,1700,2102,6666,7418,6733


In [14]:

categorical_categories.remove("device_id")
for categoria in categorical_categories:
    print(f"Los valores unicos de la categoria {categoria} son : {data[categoria].unique().tolist()}")

Los valores unicos de la categoria location son : ['Mymensingh', 'Rajshahi', 'Chattogram', 'Dhaka', 'Khulna', 'Sylhet']
Los valores unicos de la categoria crop_type son : ['Rice', 'Wheat', 'Tomato', 'Jute', 'Potato']
Los valores unicos de la categoria season son : ['winter', 'summer']
Los valores unicos de la categoria crop_health son : ['Healthy', 'Stressed', 'Critical']
Los valores unicos de la categoria pest_risk son : ['Low', 'Medium']


In [15]:
data_old = data = data.drop(columns=["device_id"])

Pasamos todas las medidas categóricas a medidas nominales haciendo OneHotEncoding en las características que tienen mas de dos valores unicos y LabelEncoding en las que solo tienen 2 valores. Lo hacemos así ya que con LabelEncoding en las de 2 valores tomarán 0 y 1 siendo un identico a OneHotEncoding dropeando una de las columnas, ya que la columa puede pasar a ser true o false.

In [16]:
data['pest_risk_encoded'] = data['pest_risk'].astype('category').cat.codes
data['season_encoded'] = data['season'].astype('category').cat.codes

# Alternativa para crop_health si entendemos que tenga orden
# crop_health_order = {'Healthy': 2, 'Stressed': 1, 'Critical': 0}
# data['crop_health_encoded'] = data['crop_health'].map(crop_health_order)
data = pd.get_dummies(data, columns=["location", "crop_type", "crop_health"])
data = data.drop(columns=["pest_risk", "season"])